In [1]:
import os
import xarray as xr
import numpy as np
from os.path import join
import geopandas as gpd
import pandas as pd
import hydromt
from hydromt import DataCatalog
from hydromt_sfincs import SfincsModel, utils

In [2]:
''' 
Script:
Author: L Grimley
Last Updated: 8/20/24

Description: 
    This script is used for plotting water level differences for the PGW ensmean for coastal, runoff, and compound drivers
    This script reads in the netCDF of zsmax and plots the difference between the future and present

Inputs:

Outputs:
- pgw_zsmax.nc
- fut_ensemble_zsmax_mean.nc
- fut_ensemble_zsmax_max.nc

'''

' \nScript:\nAuthor: L Grimley\nLast Updated: 8/20/24\n\nDescription: \n    This script is used for plotting water level differences for the PGW ensmean for coastal, runoff, and compound drivers\n    This script reads in the netCDF of zsmax and plots the difference between the future and present\n\nInputs:\n\nOutputs:\n- pgw_zsmax.nc\n- fut_ensemble_zsmax_mean.nc\n- fut_ensemble_zsmax_max.nc\n\n'

In [3]:
def get_hmax_da(mod_results_dir, depfile=None, hmin=0.05):
    da_hmax_list = []
    event_ids = []
    for dir in os.listdir(mod_results_dir):
        mod.read_results(fn_map=os.path.join(results_dir, dir, 'sfincs_map.nc'))
        zsmax = mod.results["zsmax"].max(dim='timemax')
        da_zsmax_list.append(zsmax)
        event_ids.append(dir)

        # Downscale results to get depth
        hmax = utils.downscale_floodmap(
            zsmax=zsmax,
            dep=mod.data_catalog.get_rasterdataset(depfile),
            hmin=hmin,
            gdf_mask=None,
            reproj_method='bilinear',
            floodmap_fn=os.path.join(out_dir, f'{dir}_hmax.tif'))
        da_hmax_list.append(hmax)

    da_hmax = xr.concat(da_hmax_list, dim='run')
    da_hmax['run'] = xr.IndexVariable('run', event_ids)
    return da_hmax

def calculate_ensmean_zsmax(da_zsmax, storm, climate, scenario, ntype='mean'):
    nruns = 7
    if storm == 'matt':
        nruns = 6

    # Get the list of runs in the da_zsmax so we can subset
    run_ids = da_zsmax.run.values.tolist()
    sub = [i for i in run_ids if f'{storm}_{climate}' in i]
    sub2 = [i for i in sub if f'SF{(nruns + 1)}' not in i]
    selected_ids = [i for i in sub2 if scenario in i]
    ensmean_scenario = da_zsmax.sel(run=selected_ids)
    if len(ensmean_scenario.run.values) % nruns != 0:
        print('Expected number of runs were not found...')
        print(selected_ids)
    else:
        if ntype == 'mean':
            summaryOfMembers = da_zsmax.sel(run=selected_ids).mean('run')
        elif ntype == 'max':
            summaryOfMembers = da_zsmax.sel(run=selected_ids).max('run')
    summaryOfMembers.name = f'{storm}_{climate}_{scenario}_{ntype}'

    return summaryOfMembers

In [4]:
root = r'Z:\Data-Expansion\users\lelise\projects\Carolinas_SFINCS\Chapter2_PGW\sfincs\01_AGU2023\future_florence\future_florence_ensmean'
mod = SfincsModel(root=root, mode='r')
mod.read()

In [5]:
results_dir = r'Z:\Data-Expansion\users\lelise\projects\Carolinas_SFINCS\Chapter2_PGW\sfincs\03_OBS\02_sfincs_models_future'
os.chdir(r'Z:\Data-Expansion\users\lelise\projects\Carolinas_SFINCS\Chapter2_PGW\sfincs\03_OBS\analysis_2')

out_dir = os.path.join(os.getcwd(), 'zsmax')
if os.path.exists(out_dir) is False:
    os.makedirs(out_dir)
os.chdir(out_dir)

''' Loop through all model runs and save the peak water level output to a single netcdf'''

if os.path.exists(os.path.join(out_dir, 'pgw_zsmax.nc')) is False:
    
    # Get peak water levels across all model runs
    da_zsmax_list = []
    event_ids = []

    # Loop through all the model dirs to read the results
    for dir in os.listdir(results_dir):
        try:
            # Read SFINCS model results
            mod.read_results(fn_map=os.path.join(results_dir, dir, 'sfincs_map.nc'))
            
            # Get maximum water surface elevation
            zsmax = mod.results["zsmax"].max(dim='timemax')
            
            # Append this data array to a list
            da_zsmax_list.append(zsmax)
            event_ids.append(dir)
            
        except:
            print(dir)

    # Combine data array using xarray with new dimension "run" with model name
    da_zsmax = xr.concat(da_zsmax_list, dim='run')
    da_zsmax['run'] = xr.IndexVariable('run', event_ids)
    
    # Write this to a netcdf
    da_zsmax.to_netcdf(os.path.join(out_dir, 'pgw_zsmax.nc'))
    print('Done writing zsmax for all runs!')
else:
    print('Loading previously processed data...')
    da_zsmax = xr.open_dataarray(os.path.join(out_dir, 'pgw_zsmax.nc'))

Loading previously processed data...


In [6]:
''' Calculate SFINCS ensemble mean/max water levels '''

for ntype in ['mean', 'max']:
    ensmean_list = []
    run_ids = []
    storms = ['flor', 'matt', 'floy']
    scenarios = ['coastal', 'runoff', 'compound']
    for storm in storms:
        for climate in ['fut']:
            for scenario in scenarios:
                ensmean = calculate_ensmean_zsmax(da_zsmax=da_zsmax,
                                                  storm=storm,
                                                  climate=climate,
                                                  scenario=scenario,
                                                  ntype=ntype
                                                  )
                #ensmean.raster.to_raster(f'{ensmean.name}.tif', nodata=-9999.0)
                ensmean_list.append(ensmean)
                run_ids.append(ensmean.name)
                print(ensmean.name)

    da_ensmean = xr.concat(ensmean_list, dim='run')
    da_ensmean['run'] = xr.IndexVariable('run', run_ids)
    da_ensmean.to_netcdf(f'fut_ensemble_zsmax_{ntype}.nc')

flor_fut_coastal_mean
flor_fut_runoff_mean
flor_fut_compound_mean
matt_fut_coastal_mean
matt_fut_runoff_mean
matt_fut_compound_mean
floy_fut_coastal_mean
floy_fut_runoff_mean
floy_fut_compound_mean
flor_fut_coastal_max
flor_fut_runoff_max
flor_fut_compound_max
matt_fut_coastal_max
matt_fut_runoff_max
matt_fut_compound_max
floy_fut_coastal_max
floy_fut_runoff_max
floy_fut_compound_max
